In [1]:
from helper import aoc_timer
import pandas as pd
import re
import itertools

In [2]:
class Shop:
    """Class for item shop."""

    def __init__(self, path):
        self.inventory = self.get_shop(path)

    def get_shop(self, path):
        """Read shop inventory into pandas DataFrame."""
        df = []
        for line in open(path).read().split('\n'):
            if not line:
                continue
            if ':' in line:
                # New item type (plus headers)
                idx = line.find(':')
                item_type = line[:idx]
                headers = ['Type', 'Item'] + line[idx+1:].split()
            else:
                # Item row - use RegEx to split on 2+ whitespaces
                df.append({k: v for k, v in zip(headers, [item_type] + re.split(r'\s{2,}', line))})
        return pd.DataFrame(df).astype({k: 'int8' for k in headers[2:]})


In [3]:
class Spellbook:
    """Class for magic spells."""
    
    def __init__(self, path):
        self.spells = pd.read_csv(path)


In [4]:
class Character:
    """Base class for RPG characters."""

    def __init__(self, hp=0, damage=0, armor=0, path=None):
        self.path = path  # Optionally set character attributes from input file
        if path is None:
            # Manually set attributes when initializing
            self.hp = hp
            self.damage = damage
            self.armor = armor
        else:
            # Set attributes from input file
            attr = self.get_attr(path)
            self.hp = attr.get('Hit Points', 0)
            self.damage = attr.get('Damage', 0)
            self.armor = attr.get('Armor', 0)

    def __str__(self):
        """Print base attributes."""
        attrs = {
            'HP': self.hp,
            'Damage': self.damage,
            'Armor': self.armor
        }
        return f"Class: {self.__class__.__name__}\nAttributes: {attrs}"

    def get_attr(self, path):
        """Optionally set character attributes from input file."""
        return {k: int(v) for k, v in
                [line.strip().split(': ') for line in open(path).readlines()]}

    def attack(self, char):
        """Attack given character, char."""
        hit = max(1, self.damage - char.armor)
        char.hp = max(0, char.hp - hit)

    def battle(self, char, output=True):
        """Take turns (self attacks first) to attack until one character is defeated."""
        attacker, defender = self, char
        # Battle to the death
        while self.hp and char.hp:
            attacker.attack(defender)
            attacker, defender = defender, attacker
        # Battle complete
        if self.hp:
            if output:
                print("You win!")
                print(self)
            return True
        else:
            if output:
                print("You lose!")
                print(self)
            return False


In [5]:
class Warrior(Character):
    """RPG character class that uses standard attacks during battle.
       Warrior can also purchase and equip items from the shop using gold.
       These items affect the Warrior's base attributes.
    """

    def __init__(self, hp=0, damage=0, armor=0, path=None):
        # Inherit base stats from Character class
        super().__init__(hp, damage, armor, path)
        # Warrior attributes
        self.gold = 0
        self.weapon_slots = range(1, 2)  # Must equip a weapon
        self.armor_slots = range(0, 2)   # Armor is optional
        self.ring_slots = range(0, 3)    # Maximum two rings
        # Warrior inventory
        self.weapon_equip = None
        self.armor_equip = None
        self.rings_equip = None

    def __str__(self):
        """Print attributes and inventory."""
        attrs = super().__str__()
        equip = {
            'Weapon': self.weapon_equip,
            'Armor': self.armor_equip,
            'Rings': self.rings_equip,
            'Cost': self.gold
        }
        return f"{attrs}\nInventory: {equip}"

    def purchase(self, items, shop):
        """Purchase items from the shop, amend attributes, record gold cost."""
        buy = shop.inventory.loc[items]
        self.damage = buy.Damage.sum()
        self.armor = buy.Armor.sum()
        self.gold = buy.Cost.sum()
        # Equip items
        self.weapon_equip = buy.Item.loc[buy.Type == 'Weapons'].tolist()
        self.armor_equip = buy.Item.loc[buy.Type == 'Armor'].tolist()
        self.rings_equip = buy.Item.loc[buy.Type == 'Rings'].tolist()


In [6]:
class Wizard(Character):
    """RPG character class that casts magic spells during battle.
       Wizard can cast spells from the spellbook using mana.
       If Wizard cannot afford to cast any spells, he loses.
    """

    def __init__(self, hp=0, damage=0, armor=0, mana=0, path=None):
        # Inherit base stats from Character class
        super().__init__(hp, damage, armor, path)
        # Wizard attributes
        self.mana = mana
        self._armor = armor
        # Spell effects
        self.active_effects = {}

    def __str__(self):
        attrs = super().__str__()
        equip = {
            'Mana': self.mana,
            'Effects': self.active_effects
        }
        return f"{attrs}\nInventory: {equip}"

    def cast(self, spell, spellbook):
        """Cast a spell from spellbook."""
        spell = spellbook.spells.loc[spell]
        # Check spell is castable
        if spell.Cost > self.mana:
            print(f"You can't afford to cast {spell.Spell}!  " +
                  f"It costs {spell.Cost} mana, you have {self.mana} mana.\n" +
                 "You lose!")
            self.hp = 0
            return False
        # Check spell effect is not already active
        if spell.Spell in self.active_effects:
            raise ValueError(f"Cannot cast {spell.Spell} as its effect is still active " +
                             f"with timer {self.active_effects[spell.Spell]['timer']}.")
        # Set attributes from spell
        self.mana -= spell.Cost
        self.damage = spell.Damage
        self.hp += spell.Heal
        # Set effects
        if spell.Duration:
            self.active_effects[spell.Spell] = {
                'timer': spell.Duration,
                'dot': spell.DoT,
                'aot': spell.AoT,
                'mot': spell.MoT
            }
        return True

    def attack(self, char):
        """Attack given character, char."""
        # Apply instant damage from base class
        if self.damage:
            super().attack(char)
        # Apply effects
        for name, effect in self.active_effects.items():
            char.hp = max(0, char.hp - effect['dot'])
            self.armor = self._armor + effect['aot']
            self.mana += effect['mot']
            effect['timer'] -= 1
        # Remove expired effects
        self.active_effects = {k: v for k, v in self.active_effects.items() if v['timer'] > 0}


In [7]:
# May need to implement an overall Game class which holds the characters and tracks turns
# Currently, the effects timers are not decrementing on the boss' turns, only the wizard's
# Game class could implement the battle method?

class Game():

    def __init__(self, characters):
        self.turn = 0
        self.attacker, self.defender = characters


In [8]:
@aoc_timer
def Day21(hp=100, output=False, part2=False):
    # Setup
    player = Warrior(hp, 0, 0)
    shop = Shop('shop.txt')

    # Combinations of available items
    inv = shop.inventory
    item_types = inv.Type.unique()
    combs = {k: [] for k in item_types}
    items = zip(
        [player.weapon_slots, player.armor_slots, player.ring_slots],  # Player inventory slots
        [inv.loc[inv.Type == x].index for x in item_types],            # Shop item indices
        item_types                                                     # Types of items
    )

    # Compile dictionary of all combinations of each item
    for slots, idx, item in items:
        for n in slots:
            for c in itertools.combinations(idx, n):
                combs[item].append(c)

    # Compile combinations of all possible items
    costs = {
        tuple(loadout): inv.Cost.loc[loadout].sum() for loadout in
        [list(sum(x, ())) for x in itertools.product(*combs.values())]
    }

    # Sort items by cost, purchase item combination and play game until condition is met
    for items, cost in sorted(costs.items(), key=lambda x: x[1], reverse=part2):
        # Reset characters
        player = Warrior(hp, 0, 0)
        boss = Character(path='day21.txt')
        # Purchase item combination and battle
        player.purchase(list(items), shop)
        if player.battle(boss, output=False) ^ part2:
            return cost

print("Part 1:", Day21(part2=False))
print("Part 2:", Day21(part2=True))

-----
Time: 633.8 ms
Part 1: 111
-----
Time: 781.9 ms
Part 2: 188


In [9]:
shop = Shop('shop.txt')
inv = shop.inventory
inv

,Type,Item,Cost,Damage,Armor
0,Weapons,Dagger,8,4,0
1,Weapons,Shortsword,10,5,0
2,Weapons,Warhammer,25,6,0
3,Weapons,Longsword,40,7,0
4,Weapons,Greataxe,74,8,0
5,Armor,Leather,13,0,1
6,Armor,Chainmail,31,0,2
7,Armor,Splintmail,53,0,3
8,Armor,Bandedmail,75,0,4
9,Armor,Platemail,102,0,5


In [10]:
spellbook = Spellbook('spells.txt')
spells = spellbook.spells
spells

,Spell,Cost,Damage,Heal,Duration,DoT,AoT,MoT
0,Magic Missile,53,4,0,0,0,0,0
1,Drain,73,2,2,0,0,0,0
2,Shield,113,0,0,6,0,7,0
3,Poison,173,0,0,6,3,0,0
4,Recharge,229,0,0,5,0,0,101


In [11]:
# Testing functionality
shop = Shop('shop.txt')
warrior = Warrior(hp=100)
first_boss = Character(path='day21.txt')
wizard = Wizard(hp=50, mana=500)
second_boss = Character(path='input.txt')

characters = [
    warrior,
    first_boss,
    wizard,
    second_boss
]

for char in characters:
    print(char)
    print()

warrior.purchase([3, 6, 14], shop)
warrior.battle(first_boss, output=True)

Class: Warrior
Attributes: {'HP': 100, 'Damage': 0, 'Armor': 0}
Inventory: {'Weapon': None, 'Armor': None, 'Rings': None, 'Cost': 0}

Class: Character
Attributes: {'HP': 109, 'Damage': 8, 'Armor': 2}

Class: Wizard
Attributes: {'HP': 50, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 500, 'Effects': {}}

Class: Character
Attributes: {'HP': 51, 'Damage': 9, 'Armor': 0}

You win!
Class: Warrior
Attributes: {'HP': 16, 'Damage': 7, 'Armor': 4}
Inventory: {'Weapon': ['Longsword'], 'Armor': ['Chainmail'], 'Rings': ['Defense +2'], 'Cost': 111}


True

In [12]:
# Test Wizard battle functionality
wizard = Wizard(hp=50, mana=500)
boss = Character(path='input.txt')
spellbook = Spellbook('spells.txt')
spells = spellbook.spells
print("Initial state:")
print("--------------")
print(wizard, "\n")
print(boss, "\n")
attacker, defender = wizard, boss
for t in range(10):
    spell = t % 5
    if isinstance(attacker, Wizard):
        attacker.cast(spell, spellbook)
        print(f"Wizard casts {spells.Spell.loc[spell]}!\n")
    attacker.attack(defender)
    if not isinstance(attacker, Wizard):
        # This is a hack to decrement effects timers during the boss' turns
        for effect, val in defender.active_effects.items():
            val['timer'] -= 1
        print(f"{attacker.__class__.__name__} attacks {defender.__class__.__name__}!\n")
    print("New state:")
    print("----------")
    print(attacker, "\n")
    print(defender, "\n")
    attacker, defender = defender, attacker


Initial state:
--------------
Class: Wizard
Attributes: {'HP': 50, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 500, 'Effects': {}} 

Class: Character
Attributes: {'HP': 51, 'Damage': 9, 'Armor': 0} 

Wizard casts Magic Missile!

New state:
----------
Class: Wizard
Attributes: {'HP': 50, 'Damage': 4, 'Armor': 0}
Inventory: {'Mana': 447, 'Effects': {}} 

Class: Character
Attributes: {'HP': 47, 'Damage': 9, 'Armor': 0} 

Character attacks Wizard!

New state:
----------
Class: Character
Attributes: {'HP': 47, 'Damage': 9, 'Armor': 0} 

Class: Wizard
Attributes: {'HP': 41, 'Damage': 4, 'Armor': 0}
Inventory: {'Mana': 447, 'Effects': {}} 

Wizard casts Shield!

New state:
----------
Class: Wizard
Attributes: {'HP': 41, 'Damage': 0, 'Armor': 7}
Inventory: {'Mana': 334, 'Effects': {'Shield': {'timer': 5, 'dot': 0, 'aot': 7, 'mot': 0}}} 

Class: Character
Attributes: {'HP': 47, 'Damage': 9, 'Armor': 0} 

Character attacks Wizard!

New state:
----------
Class: Character
Attributes: {'HP': 47, '

In [15]:
"""
For example, suppose the player has 10 hit points and 250 mana, and that the boss has 13 hit points and 8 damage:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
"""
# Test Wizard battle functionality
wizard = Wizard(hp=10, mana=250)
boss = Character(hp=13, damage=8)
spellbook = Spellbook('spells.txt')
spells = spellbook.spells
spell_order = [3, 0]
print("Initial state:")
print("--------------")
print(wizard, "\n")
print(boss, "\n")
attacker, defender = wizard, boss
for t in range(4):
    spell = spell_order[t // 2]
    if isinstance(attacker, Wizard):
        attacker.cast(spell, spellbook)
        print(f"Wizard casts {spells.Spell.loc[spell]}!\n")
    attacker.attack(defender)
    if not isinstance(attacker, Wizard):
        # This is a hack to decrement effects timers during the boss' turns
        for effect, val in defender.active_effects.items():
            val['timer'] -= 1
        print(f"{attacker.__class__.__name__} attacks {defender.__class__.__name__}!\n")
    print("New state:")
    print("----------")
    print(attacker, "\n")
    print(defender, "\n")
    attacker, defender = defender, attacker

# DoTs aren't applying on boss' turn - need to encapsulate this into battle or into Game class


Initial state:
--------------
Class: Wizard
Attributes: {'HP': 10, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 250, 'Effects': {}} 

Class: Character
Attributes: {'HP': 13, 'Damage': 8, 'Armor': 0} 

Wizard casts Poison!

New state:
----------
Class: Wizard
Attributes: {'HP': 10, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 77, 'Effects': {'Poison': {'timer': 5, 'dot': 3, 'aot': 0, 'mot': 0}}} 

Class: Character
Attributes: {'HP': 10, 'Damage': 8, 'Armor': 0} 

Character attacks Wizard!

New state:
----------
Class: Character
Attributes: {'HP': 10, 'Damage': 8, 'Armor': 0} 

Class: Wizard
Attributes: {'HP': 2, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 77, 'Effects': {'Poison': {'timer': 4, 'dot': 3, 'aot': 0, 'mot': 0}}} 

Wizard casts Magic Missile!

New state:
----------
Class: Wizard
Attributes: {'HP': 2, 'Damage': 4, 'Armor': 0}
Inventory: {'Mana': 24, 'Effects': {'Poison': {'timer': 3, 'dot': 3, 'aot': 0, 'mot': 0}}} 

Class: Character
Attributes: {'HP': 3, 'Damage': 8, 'Armor